Aidan Coyle

afcoyle@uw.edu

2021-07-01

Roberts Lab, UW-SAFS

IMPORTANT: Unlike the manual clusters for cbai_v4.0 and hemat_v1.6, the current bar for expression in these transcriptome's clusters is 10 counts total over all samples for that crab. For the other transcriptomes, it is 5 counts. This is because the number of counts for cbai_v2.0 stops a function used in heatmap creation from running in the previous script.

In script 7_1_manual_clustering_cbaiv2.0.Rmd, we took libraries aligned to an unfiltered transcriptome (cbai_transcriptomev2.0), grouped them according to crab (e.g. took all libraries for Crab A, B, C...) and clustered gene expression into modules based on expression patterns

We then described the expression patterns of each module as following one of five patterns. Crabs with three time points (ambient- and lowered-temperature treatment crab) had the following notation used:

- High to low (HTL): Expression decreases over time (regardless of whether the decrease took place on Day 2 or Day 17)

- Low to high (LTH): Expression increases over time (regardless of whether the increase took place on Day 2 or Day 17)

- Low High Low (LHL): Expression increases on Day 2, and then drops on Day 17

- High Low High (HLH): Expression drops on Day 2 and then increases on Day 17

- Mixed (MIX): Expression within the module follows no clear pattern

Crabs in the Elevated-temperature treatment group had only two time points (crabs G, H, and I). For these, a different notation was used. 

- LL = expression stays low

- HH = expression stays high

- LH = expression goes from low to high

- HL = expression goes from high to low

- MIX = mixed - no clear pattern of expression within the module

Importantly, **multiple modules within a single crab could be given the same assignment**. This issue is what this script is meant to solve by merging gene lists.

First, let's see an example of one crab

In [1]:
!ls ../output/manual_clustering/cbai_transcriptomev2.0/Crab_A/

cluster_HLH.txt		  cluster_LHL.txt	    cluster_MIX.txt
cluster_HLH_heatmap.png   cluster_LHL_heatmap.png   cluster_MIX_heatmap.png
cluster_HTL.txt		  cluster_LTH.txt	    heatmap.png
cluster_HTL2.txt	  cluster_LTH2.txt	    manual_clustnums
cluster_HTL2_heatmap.png  cluster_LTH2_heatmap.png  merged_modules
cluster_HTL_heatmap.png   cluster_LTH_heatmap.png


And let's also see what each cluster looks like

In [2]:
!head ../output/manual_clustering/cbai_transcriptomev2.0/Crab_A/cluster_HLH.txt

"id178_TPM"	"id359_TPM"	"id463_TPM"
"TRINITY_DN27933_c0_g1_i1"	4.26297	2.75102	3.72238
"TRINITY_DN44127_c2_g1_i1"	5.7006	1.23889	7.34844
"TRINITY_DN1172_c38_g3_i3"	7.18434	1.4249	7.88532
"TRINITY_DN15444_c0_g1_i2"	4.30782	2.5026	3.74966
"TRINITY_DN60211_c0_g1_i14"	12.5478	5.78291	10.393
"TRINITY_DN14279_c0_g1_i14"	7.28032	4.37067	6.3788
"TRINITY_DN226_c5_g1_i16"	5.41642	2.80646	4.54258
"TRINITY_DN5229_c2_g1_i1"	11.0762	0	12.3603
"TRINITY_DN40265_c0_g1_i2"	9.12058	1.99923	12.5864


Looks like we need to remove the first line of each file - otherwise, when we merge modules, the header line will be included. And since columns correspond to days 0, 2, and 17 samples, it's not too meaningful

Now, let's see how many crab folders we have

In [3]:
!ls ../output/manual_clustering/cbai_transcriptomev2.0/

Crab_A	Crab_D	Crab_G	merged_modules_percentages.csv
Crab_B	Crab_E	Crab_H	merged_modules_raw_counts.csv
Crab_C	Crab_F	Crab_I


Looks good! We can move on.

## Crab A

We'll now start on merging all modules for Crab A

Let's take another look at the current modules for Crab A

In [4]:
!ls ../output/manual_clustering/cbai_transcriptomev2.0/Crab_A/

cluster_HLH.txt		  cluster_LHL.txt	    cluster_MIX.txt
cluster_HLH_heatmap.png   cluster_LHL_heatmap.png   cluster_MIX_heatmap.png
cluster_HTL.txt		  cluster_LTH.txt	    heatmap.png
cluster_HTL2.txt	  cluster_LTH2.txt	    manual_clustnums
cluster_HTL2_heatmap.png  cluster_LTH2_heatmap.png  merged_modules
cluster_HTL_heatmap.png   cluster_LTH_heatmap.png


In [11]:
# Make new directory for merged modules
!mkdir ../output/manual_clustering/cbai_transcriptomev2.0/Crab_A/merged_modules

# Merge all HTL modules
!find ../output/manual_clustering/cbai_transcriptomev2.0/Crab_A -maxdepth 1 -name cluster_HTL*txt | xargs -n 1 tail -n +2 > ../output/manual_clustering/cbai_transcriptomev2.0/Crab_A/merged_modules/HTL_merged.txt

# Merge all LTH modules
!find ../output/manual_clustering/cbai_transcriptomev2.0/Crab_A -maxdepth 1 -name cluster_LTH*txt | xargs -n 1 tail -n +2 > ../output/manual_clustering/cbai_transcriptomev2.0/Crab_A/merged_modules/LTH_merged.txt

# Merge all HLH modules
!find ../output/manual_clustering/cbai_transcriptomev2.0/Crab_A -maxdepth 1 -name cluster_HLH*txt | xargs -n 1 tail -n +2 > ../output/manual_clustering/cbai_transcriptomev2.0/Crab_A/merged_modules/HLH_merged.txt

# Merge all LHL modules
!find ../output/manual_clustering/cbai_transcriptomev2.0/Crab_A -maxdepth 1 -name cluster_LHL*txt | xargs -n 1 tail -n +2 > ../output/manual_clustering/cbai_transcriptomev2.0/Crab_A/merged_modules/LHL_merged.txt

# Merge all MIX modules
!find ../output/manual_clustering/cbai_transcriptomev2.0/Crab_A -maxdepth 1 -name cluster_MIX*txt | xargs -n 1 tail -n +2 > ../output/manual_clustering/cbai_transcriptomev2.0/Crab_A/merged_modules/MIX_merged.txt


Check we did this right by examining number of lines. There will be slightly fewer in merged_modules, as we removed headers

In [12]:
!wc -l ../output/manual_clustering/cbai_transcriptomev2.0/Crab_A/cluster_*txt

     38 ../output/manual_clustering/cbai_transcriptomev2.0/Crab_A/cluster_HLH.txt
    779 ../output/manual_clustering/cbai_transcriptomev2.0/Crab_A/cluster_HTL.txt
   4814 ../output/manual_clustering/cbai_transcriptomev2.0/Crab_A/cluster_HTL2.txt
   3334 ../output/manual_clustering/cbai_transcriptomev2.0/Crab_A/cluster_LHL.txt
  26543 ../output/manual_clustering/cbai_transcriptomev2.0/Crab_A/cluster_LTH.txt
    581 ../output/manual_clustering/cbai_transcriptomev2.0/Crab_A/cluster_LTH2.txt
     65 ../output/manual_clustering/cbai_transcriptomev2.0/Crab_A/cluster_MIX.txt
  36154 total


In [13]:
!wc -l ../output/manual_clustering/cbai_transcriptomev2.0/Crab_A/merged_modules/*merged.txt

     37 ../output/manual_clustering/cbai_transcriptomev2.0/Crab_A/merged_modules/HLH_merged.txt
   5591 ../output/manual_clustering/cbai_transcriptomev2.0/Crab_A/merged_modules/HTL_merged.txt
   3333 ../output/manual_clustering/cbai_transcriptomev2.0/Crab_A/merged_modules/LHL_merged.txt
  27122 ../output/manual_clustering/cbai_transcriptomev2.0/Crab_A/merged_modules/LTH_merged.txt
     64 ../output/manual_clustering/cbai_transcriptomev2.0/Crab_A/merged_modules/MIX_merged.txt
  36147 total


Looks good! We can move on.

## Crab B

In [14]:
!ls ../output/manual_clustering/cbai_transcriptomev2.0/Crab_B/

cluster_HLH.txt		  cluster_HTL3_heatmap.png  cluster_LTH.txt
cluster_HLH_heatmap.png   cluster_HTL_heatmap.png   cluster_LTH_heatmap.png
cluster_HTL.txt		  cluster_LHL.txt	    heatmap.png
cluster_HTL2.txt	  cluster_LHL2.txt	    manual_clustnums
cluster_HTL2_heatmap.png  cluster_LHL2_heatmap.png  merged_modules
cluster_HTL3.txt	  cluster_LHL_heatmap.png


In [15]:
# Make new directory for merged modules
!mkdir ../output/manual_clustering/cbai_transcriptomev2.0/Crab_B/merged_modules

# Merge all HTL modules
!find ../output/manual_clustering/cbai_transcriptomev2.0/Crab_B -maxdepth 1 -name cluster_HTL*txt | xargs -n 1 tail -n +2 > ../output/manual_clustering/cbai_transcriptomev2.0/Crab_B/merged_modules/HTL_merged.txt

# Merge all LTH modules
!find ../output/manual_clustering/cbai_transcriptomev2.0/Crab_B -maxdepth 1 -name cluster_LTH*txt | xargs -n 1 tail -n +2 > ../output/manual_clustering/cbai_transcriptomev2.0/Crab_B/merged_modules/LTH_merged.txt

# Merge all HLH modules
!find ../output/manual_clustering/cbai_transcriptomev2.0/Crab_B -maxdepth 1 -name cluster_HLH*txt | xargs -n 1 tail -n +2 > ../output/manual_clustering/cbai_transcriptomev2.0/Crab_B/merged_modules/HLH_merged.txt

# Merge all LHL modules
!find ../output/manual_clustering/cbai_transcriptomev2.0/Crab_B -maxdepth 1 -name cluster_LHL*txt | xargs -n 1 tail -n +2 > ../output/manual_clustering/cbai_transcriptomev2.0/Crab_B/merged_modules/LHL_merged.txt

# Won't merge MIX modules, as none are present in this crab

Check we did this right by examining number of lines. There will be slightly fewer in merged_modules, as we removed headers

In [16]:
!wc -l ../output/manual_clustering/cbai_transcriptomev2.0/Crab_B/cluster_*txt

  1290 ../output/manual_clustering/cbai_transcriptomev2.0/Crab_B/cluster_HLH.txt
  4256 ../output/manual_clustering/cbai_transcriptomev2.0/Crab_B/cluster_HTL.txt
   103 ../output/manual_clustering/cbai_transcriptomev2.0/Crab_B/cluster_HTL2.txt
   178 ../output/manual_clustering/cbai_transcriptomev2.0/Crab_B/cluster_HTL3.txt
  9789 ../output/manual_clustering/cbai_transcriptomev2.0/Crab_B/cluster_LHL.txt
   260 ../output/manual_clustering/cbai_transcriptomev2.0/Crab_B/cluster_LHL2.txt
  4630 ../output/manual_clustering/cbai_transcriptomev2.0/Crab_B/cluster_LTH.txt
 20506 total


In [17]:
!wc -l ../output/manual_clustering/cbai_transcriptomev2.0/Crab_B/merged_modules/*merged.txt

  1289 ../output/manual_clustering/cbai_transcriptomev2.0/Crab_B/merged_modules/HLH_merged.txt
  4534 ../output/manual_clustering/cbai_transcriptomev2.0/Crab_B/merged_modules/HTL_merged.txt
 10047 ../output/manual_clustering/cbai_transcriptomev2.0/Crab_B/merged_modules/LHL_merged.txt
  4629 ../output/manual_clustering/cbai_transcriptomev2.0/Crab_B/merged_modules/LTH_merged.txt
 20499 total


Looks good! We can move on.

## Crab C

In [18]:
!ls ../output/manual_clustering/cbai_transcriptomev2.0/Crab_C/

cluster_HLH.txt		  cluster_LHL.txt	    cluster_LTH2_heatmap.png
cluster_HLH_heatmap.png   cluster_LHL2.txt	    cluster_LTH_heatmap.png
cluster_HTL.txt		  cluster_LHL2_heatmap.png  heatmap.png
cluster_HTL2.txt	  cluster_LHL_heatmap.png   manual_clustnums
cluster_HTL2_heatmap.png  cluster_LTH.txt
cluster_HTL_heatmap.png   cluster_LTH2.txt


In [19]:
# Make new directory for merged modules
!mkdir ../output/manual_clustering/cbai_transcriptomev2.0/Crab_C/merged_modules

# Merge all HTL modules
!find ../output/manual_clustering/cbai_transcriptomev2.0/Crab_C -maxdepth 1 -name cluster_HTL*txt | xargs -n 1 tail -n +2 > ../output/manual_clustering/cbai_transcriptomev2.0/Crab_C/merged_modules/HTL_merged.txt

# Merge all LTH modules
!find ../output/manual_clustering/cbai_transcriptomev2.0/Crab_C -maxdepth 1 -name cluster_LTH*txt | xargs -n 1 tail -n +2 > ../output/manual_clustering/cbai_transcriptomev2.0/Crab_C/merged_modules/LTH_merged.txt

# Merge all HLH modules
!find ../output/manual_clustering/cbai_transcriptomev2.0/Crab_C -maxdepth 1 -name cluster_HLH*txt | xargs -n 1 tail -n +2 > ../output/manual_clustering/cbai_transcriptomev2.0/Crab_C/merged_modules/HLH_merged.txt

# Merge all LHL modules
!find ../output/manual_clustering/cbai_transcriptomev2.0/Crab_C -maxdepth 1 -name cluster_LHL*txt | xargs -n 1 tail -n +2 > ../output/manual_clustering/cbai_transcriptomev2.0/Crab_C/merged_modules/LHL_merged.txt

# Won't merge MIX modules, as none are present in this crab

Check we did this right by examining number of lines. There will be slightly fewer in merged_modules, as we removed headers

In [20]:
!wc -l ../output/manual_clustering/cbai_transcriptomev2.0/Crab_C/cluster_*txt

  2365 ../output/manual_clustering/cbai_transcriptomev2.0/Crab_C/cluster_HLH.txt
  1876 ../output/manual_clustering/cbai_transcriptomev2.0/Crab_C/cluster_HTL.txt
  3088 ../output/manual_clustering/cbai_transcriptomev2.0/Crab_C/cluster_HTL2.txt
  1235 ../output/manual_clustering/cbai_transcriptomev2.0/Crab_C/cluster_LHL.txt
  1423 ../output/manual_clustering/cbai_transcriptomev2.0/Crab_C/cluster_LHL2.txt
  2762 ../output/manual_clustering/cbai_transcriptomev2.0/Crab_C/cluster_LTH.txt
   583 ../output/manual_clustering/cbai_transcriptomev2.0/Crab_C/cluster_LTH2.txt
 13332 total


In [21]:
!wc -l ../output/manual_clustering/cbai_transcriptomev2.0/Crab_C/merged_modules/*merged.txt

  2364 ../output/manual_clustering/cbai_transcriptomev2.0/Crab_C/merged_modules/HLH_merged.txt
  4962 ../output/manual_clustering/cbai_transcriptomev2.0/Crab_C/merged_modules/HTL_merged.txt
  2656 ../output/manual_clustering/cbai_transcriptomev2.0/Crab_C/merged_modules/LHL_merged.txt
  3343 ../output/manual_clustering/cbai_transcriptomev2.0/Crab_C/merged_modules/LTH_merged.txt
 13325 total


Looks good! We can move on.

## Crab D

We'll now start on merging all modules for Crab D

Let's take another look at the current modules for Crab D

In [22]:
!ls ../output/manual_clustering/cbai_transcriptomev2.0/Crab_D/

cluster_HLH.txt		  cluster_HTL_heatmap.png   cluster_LTH.txt
cluster_HLH_heatmap.png   cluster_LHL.txt	    cluster_LTH2.txt
cluster_HTL.txt		  cluster_LHL2.txt	    cluster_LTH2_heatmap.png
cluster_HTL2.txt	  cluster_LHL2_heatmap.png  cluster_LTH_heatmap.png
cluster_HTL2_heatmap.png  cluster_LHL_heatmap.png   heatmap.png


In [23]:
# Make new directory for merged modules
!mkdir ../output/manual_clustering/cbai_transcriptomev2.0/Crab_D/merged_modules

# Merge all HTL modules
!find ../output/manual_clustering/cbai_transcriptomev2.0/Crab_D -maxdepth 1 -name cluster_HTL*txt | xargs -n 1 tail -n +2 > ../output/manual_clustering/cbai_transcriptomev2.0/Crab_D/merged_modules/HTL_merged.txt

# Merge all LTH modules
!find ../output/manual_clustering/cbai_transcriptomev2.0/Crab_D -maxdepth 1 -name cluster_LTH*txt | xargs -n 1 tail -n +2 > ../output/manual_clustering/cbai_transcriptomev2.0/Crab_D/merged_modules/LTH_merged.txt

# Merge all HLH modules
!find ../output/manual_clustering/cbai_transcriptomev2.0/Crab_D -maxdepth 1 -name cluster_HLH*txt | xargs -n 1 tail -n +2 > ../output/manual_clustering/cbai_transcriptomev2.0/Crab_D/merged_modules/HLH_merged.txt

# Merge all LHL modules
!find ../output/manual_clustering/cbai_transcriptomev2.0/Crab_D -maxdepth 1 -name cluster_LHL*txt | xargs -n 1 tail -n +2 > ../output/manual_clustering/cbai_transcriptomev2.0/Crab_D/merged_modules/LHL_merged.txt

# Won't merge MIX modules, as none are present in this crab

Check we did this right by examining number of lines. There will be slightly fewer in merged_modules, as we removed headers

In [24]:
!wc -l ../output/manual_clustering/cbai_transcriptomev2.0/Crab_D/cluster_*txt

   6619 ../output/manual_clustering/cbai_transcriptomev2.0/Crab_D/cluster_HLH.txt
   9138 ../output/manual_clustering/cbai_transcriptomev2.0/Crab_D/cluster_HTL.txt
   3658 ../output/manual_clustering/cbai_transcriptomev2.0/Crab_D/cluster_HTL2.txt
   4994 ../output/manual_clustering/cbai_transcriptomev2.0/Crab_D/cluster_LHL.txt
    798 ../output/manual_clustering/cbai_transcriptomev2.0/Crab_D/cluster_LHL2.txt
   4708 ../output/manual_clustering/cbai_transcriptomev2.0/Crab_D/cluster_LTH.txt
    124 ../output/manual_clustering/cbai_transcriptomev2.0/Crab_D/cluster_LTH2.txt
  30039 total


In [25]:
!wc -l ../output/manual_clustering/cbai_transcriptomev2.0/Crab_D/merged_modules/*merged.txt

   6618 ../output/manual_clustering/cbai_transcriptomev2.0/Crab_D/merged_modules/HLH_merged.txt
  12794 ../output/manual_clustering/cbai_transcriptomev2.0/Crab_D/merged_modules/HTL_merged.txt
   5790 ../output/manual_clustering/cbai_transcriptomev2.0/Crab_D/merged_modules/LHL_merged.txt
   4830 ../output/manual_clustering/cbai_transcriptomev2.0/Crab_D/merged_modules/LTH_merged.txt
  30032 total


Looks good! We can move on.

## Crab E

In [26]:
!ls ../output/manual_clustering/cbai_transcriptomev2.0/Crab_E/

cluster_HLH.txt		  cluster_LHL.txt	    cluster_LTH2_heatmap.png
cluster_HLH_heatmap.png   cluster_LHL2.txt	    cluster_LTH_heatmap.png
cluster_HTL.txt		  cluster_LHL2_heatmap.png  heatmap.png
cluster_HTL2.txt	  cluster_LHL_heatmap.png   manual_clustnums
cluster_HTL2_heatmap.png  cluster_LTH.txt
cluster_HTL_heatmap.png   cluster_LTH2.txt


In [27]:
# Make new directory for merged modules
!mkdir ../output/manual_clustering/cbai_transcriptomev2.0/Crab_E/merged_modules

# Merge all HTL modules
!find ../output/manual_clustering/cbai_transcriptomev2.0/Crab_E -maxdepth 1 -name cluster_HTL*txt | xargs -n 1 tail -n +2 > ../output/manual_clustering/cbai_transcriptomev2.0/Crab_E/merged_modules/HTL_merged.txt

# Merge all LTH modules
!find ../output/manual_clustering/cbai_transcriptomev2.0/Crab_E -maxdepth 1 -name cluster_LTH*txt | xargs -n 1 tail -n +2 > ../output/manual_clustering/cbai_transcriptomev2.0/Crab_E/merged_modules/LTH_merged.txt

# Merge all HLH modules
!find ../output/manual_clustering/cbai_transcriptomev2.0/Crab_E -maxdepth 1 -name cluster_HLH*txt | xargs -n 1 tail -n +2 > ../output/manual_clustering/cbai_transcriptomev2.0/Crab_E/merged_modules/HLH_merged.txt

# Merge all LHL modules
!find ../output/manual_clustering/cbai_transcriptomev2.0/Crab_E -maxdepth 1 -name cluster_LHL*txt | xargs -n 1 tail -n +2 > ../output/manual_clustering/cbai_transcriptomev2.0/Crab_E/merged_modules/LHL_merged.txt

# Won't merge MIX modules, as none are present in this crab

Check we did this right by examining number of lines. There will be slightly fewer in merged_modules, as we removed headers

In [28]:
!wc -l ../output/manual_clustering/cbai_transcriptomev2.0/Crab_E/cluster_*txt

  1522 ../output/manual_clustering/cbai_transcriptomev2.0/Crab_E/cluster_HLH.txt
  5457 ../output/manual_clustering/cbai_transcriptomev2.0/Crab_E/cluster_HTL.txt
  1749 ../output/manual_clustering/cbai_transcriptomev2.0/Crab_E/cluster_HTL2.txt
  2544 ../output/manual_clustering/cbai_transcriptomev2.0/Crab_E/cluster_LHL.txt
  1519 ../output/manual_clustering/cbai_transcriptomev2.0/Crab_E/cluster_LHL2.txt
   299 ../output/manual_clustering/cbai_transcriptomev2.0/Crab_E/cluster_LTH.txt
  3410 ../output/manual_clustering/cbai_transcriptomev2.0/Crab_E/cluster_LTH2.txt
 16500 total


In [29]:
!wc -l ../output/manual_clustering/cbai_transcriptomev2.0/Crab_E/merged_modules/*merged.txt

  1521 ../output/manual_clustering/cbai_transcriptomev2.0/Crab_E/merged_modules/HLH_merged.txt
  7204 ../output/manual_clustering/cbai_transcriptomev2.0/Crab_E/merged_modules/HTL_merged.txt
  4061 ../output/manual_clustering/cbai_transcriptomev2.0/Crab_E/merged_modules/LHL_merged.txt
  3707 ../output/manual_clustering/cbai_transcriptomev2.0/Crab_E/merged_modules/LTH_merged.txt
 16493 total


Looks good! We can move on.

## Crab F

We'll now start on merging all modules for Crab F

Let's take another look at the current modules for Crab F

In [30]:
!ls ../output/manual_clustering/cbai_transcriptomev2.0/Crab_F/

cluster_HLH.txt		  cluster_HTL_heatmap.png  cluster_LTH2_heatmap.png
cluster_HLH_heatmap.png   cluster_LHL.txt	   cluster_LTH_heatmap.png
cluster_HTL.txt		  cluster_LHL_heatmap.png  heatmap.png
cluster_HTL2.txt	  cluster_LTH.txt
cluster_HTL2_heatmap.png  cluster_LTH2.txt


In [31]:
# Make new directory for merged modules
!mkdir ../output/manual_clustering/cbai_transcriptomev2.0/Crab_F/merged_modules

# Merge all HTL modules
!find ../output/manual_clustering/cbai_transcriptomev2.0/Crab_F -maxdepth 1 -name cluster_HTL*txt | xargs -n 1 tail -n +2 > ../output/manual_clustering/cbai_transcriptomev2.0/Crab_F/merged_modules/HTL_merged.txt

# Merge all LTH modules
!find ../output/manual_clustering/cbai_transcriptomev2.0/Crab_F -maxdepth 1 -name cluster_LTH*txt | xargs -n 1 tail -n +2 > ../output/manual_clustering/cbai_transcriptomev2.0/Crab_F/merged_modules/LTH_merged.txt

# Merge all HLH modules
!find ../output/manual_clustering/cbai_transcriptomev2.0/Crab_F -maxdepth 1 -name cluster_HLH*txt | xargs -n 1 tail -n +2 > ../output/manual_clustering/cbai_transcriptomev2.0/Crab_F/merged_modules/HLH_merged.txt

# Merge all LHL modules
!find ../output/manual_clustering/cbai_transcriptomev2.0/Crab_F -maxdepth 1 -name cluster_LHL*txt | xargs -n 1 tail -n +2 > ../output/manual_clustering/cbai_transcriptomev2.0/Crab_F/merged_modules/LHL_merged.txt

# Won't merge MIX modules, as none are present in this crab

Check we did this right by examining number of lines. There will be slightly fewer in merged_modules, as we removed headers

In [32]:
!wc -l ../output/manual_clustering/cbai_transcriptomev2.0/Crab_F/cluster_*txt

    85 ../output/manual_clustering/cbai_transcriptomev2.0/Crab_F/cluster_HLH.txt
  5306 ../output/manual_clustering/cbai_transcriptomev2.0/Crab_F/cluster_HTL.txt
  1931 ../output/manual_clustering/cbai_transcriptomev2.0/Crab_F/cluster_HTL2.txt
   395 ../output/manual_clustering/cbai_transcriptomev2.0/Crab_F/cluster_LHL.txt
  5819 ../output/manual_clustering/cbai_transcriptomev2.0/Crab_F/cluster_LTH.txt
   294 ../output/manual_clustering/cbai_transcriptomev2.0/Crab_F/cluster_LTH2.txt
 13830 total


In [33]:
!wc -l ../output/manual_clustering/cbai_transcriptomev2.0/Crab_F/merged_modules/*merged.txt

    84 ../output/manual_clustering/cbai_transcriptomev2.0/Crab_F/merged_modules/HLH_merged.txt
  7235 ../output/manual_clustering/cbai_transcriptomev2.0/Crab_F/merged_modules/HTL_merged.txt
   394 ../output/manual_clustering/cbai_transcriptomev2.0/Crab_F/merged_modules/LHL_merged.txt
  6111 ../output/manual_clustering/cbai_transcriptomev2.0/Crab_F/merged_modules/LTH_merged.txt
 13824 total


Looks good! We can move on.

## Crab G

In [34]:
!ls ../output/manual_clustering/cbai_transcriptomev2.0/Crab_G/

cluster_HL.txt		 cluster_LH2.txt	  cluster_MIX.txt
cluster_HL2.txt		 cluster_LH2_heatmap.png  cluster_MIX_heatmap.png
cluster_HL2_heatmap.png  cluster_LH_heatmap.png   heatmap.png
cluster_HL_heatmap.png	 cluster_LL.txt		  manual_clustnums
cluster_LH.txt		 cluster_LL_heatmap.png


In [35]:
# Make new directory for merged modules
!mkdir ../output/manual_clustering/cbai_transcriptomev2.0/Crab_G/merged_modules

# Merge all HL modules
!find ../output/manual_clustering/cbai_transcriptomev2.0/Crab_G -maxdepth 1 -name cluster_HL*txt | xargs -n 1 tail -n +2 > ../output/manual_clustering/cbai_transcriptomev2.0/Crab_G/merged_modules/HL_merged.txt

# Merge all LH modules
!find ../output/manual_clustering/cbai_transcriptomev2.0/Crab_G -maxdepth 1 -name cluster_LH*txt | xargs -n 1 tail -n +2 > ../output/manual_clustering/cbai_transcriptomev2.0/Crab_G/merged_modules/LH_merged.txt

# Merge all LL modules
!find ../output/manual_clustering/cbai_transcriptomev2.0/Crab_G -maxdepth 1 -name cluster_LL*txt | xargs -n 1 tail -n +2 > ../output/manual_clustering/cbai_transcriptomev2.0/Crab_G/merged_modules/LL_merged.txt

# Merge all MIX modules
!find ../output/manual_clustering/cbai_transcriptomev2.0/Crab_G -maxdepth 1 -name cluster_MIX*txt | xargs -n 1 tail -n +2 > ../output/manual_clustering/cbai_transcriptomev2.0/Crab_G/merged_modules/MIX_merged.txt

# Won't merge HH modules, as none are present in this crab

Check we did this right by examining number of lines. There will be slightly fewer in merged_modules, as we removed headers

In [36]:
!wc -l ../output/manual_clustering/cbai_transcriptomev2.0/Crab_G/cluster_*txt

  8021 ../output/manual_clustering/cbai_transcriptomev2.0/Crab_G/cluster_HL.txt
   646 ../output/manual_clustering/cbai_transcriptomev2.0/Crab_G/cluster_HL2.txt
  7737 ../output/manual_clustering/cbai_transcriptomev2.0/Crab_G/cluster_LH.txt
   113 ../output/manual_clustering/cbai_transcriptomev2.0/Crab_G/cluster_LH2.txt
   313 ../output/manual_clustering/cbai_transcriptomev2.0/Crab_G/cluster_LL.txt
    25 ../output/manual_clustering/cbai_transcriptomev2.0/Crab_G/cluster_MIX.txt
 16855 total


In [37]:
!wc -l ../output/manual_clustering/cbai_transcriptomev2.0/Crab_G/merged_modules/*merged.txt

  8665 ../output/manual_clustering/cbai_transcriptomev2.0/Crab_G/merged_modules/HL_merged.txt
  7848 ../output/manual_clustering/cbai_transcriptomev2.0/Crab_G/merged_modules/LH_merged.txt
   312 ../output/manual_clustering/cbai_transcriptomev2.0/Crab_G/merged_modules/LL_merged.txt
    24 ../output/manual_clustering/cbai_transcriptomev2.0/Crab_G/merged_modules/MIX_merged.txt
 16849 total


Looks good! We can move on.

## Crab H

In [38]:
!ls ../output/manual_clustering/cbai_transcriptomev2.0/Crab_H/

cluster_HL.txt		cluster_LH_heatmap.png	 heatmap.png
cluster_HL_heatmap.png	cluster_MIX.txt		 manual_clustnums
cluster_LH.txt		cluster_MIX_heatmap.png


In [39]:
# Make new directory for merged modules
!mkdir ../output/manual_clustering/cbai_transcriptomev2.0/Crab_H/merged_modules

# Merge all HL modules
!find ../output/manual_clustering/cbai_transcriptomev2.0/Crab_H -maxdepth 1 -name cluster_HL*txt | xargs -n 1 tail -n +2 > ../output/manual_clustering/cbai_transcriptomev2.0/Crab_H/merged_modules/HL_merged.txt

# Merge all LH modules
!find ../output/manual_clustering/cbai_transcriptomev2.0/Crab_H -maxdepth 1 -name cluster_LH*txt | xargs -n 1 tail -n +2 > ../output/manual_clustering/cbai_transcriptomev2.0/Crab_H/merged_modules/LH_merged.txt

# Merge all MIX modules
!find ../output/manual_clustering/cbai_transcriptomev2.0/Crab_H -maxdepth 1 -name cluster_MIX*txt | xargs -n 1 tail -n +2 > ../output/manual_clustering/cbai_transcriptomev2.0/Crab_H/merged_modules/MIX_merged.txt

# Won't merge HH or LL modules, as none are present in this crab

Check we did this right by examining number of lines. There will be slightly fewer in merged_modules, as we removed headers

In [40]:
!wc -l ../output/manual_clustering/cbai_transcriptomev2.0/Crab_H/cluster_*txt

  6457 ../output/manual_clustering/cbai_transcriptomev2.0/Crab_H/cluster_HL.txt
  2206 ../output/manual_clustering/cbai_transcriptomev2.0/Crab_H/cluster_LH.txt
    12 ../output/manual_clustering/cbai_transcriptomev2.0/Crab_H/cluster_MIX.txt
  8675 total


In [41]:
!wc -l ../output/manual_clustering/cbai_transcriptomev2.0/Crab_H/merged_modules/*merged.txt

  6456 ../output/manual_clustering/cbai_transcriptomev2.0/Crab_H/merged_modules/HL_merged.txt
  2205 ../output/manual_clustering/cbai_transcriptomev2.0/Crab_H/merged_modules/LH_merged.txt
    11 ../output/manual_clustering/cbai_transcriptomev2.0/Crab_H/merged_modules/MIX_merged.txt
  8672 total


Looks good! We can move on.

## Crab I

In [42]:
!ls ../output/manual_clustering/cbai_transcriptomev2.0/Crab_I/

cluster_HL.txt		cluster_LH_heatmap.png	 heatmap.png
cluster_HL_heatmap.png	cluster_MIX.txt		 manual_clustnums
cluster_LH.txt		cluster_MIX_heatmap.png


In [43]:
# Make new directory for merged modules
!mkdir ../output/manual_clustering/cbai_transcriptomev2.0/Crab_I/merged_modules

# Merge all HL modules
!find ../output/manual_clustering/cbai_transcriptomev2.0/Crab_I -maxdepth 1 -name cluster_HL*txt | xargs -n 1 tail -n +2 > ../output/manual_clustering/cbai_transcriptomev2.0/Crab_I/merged_modules/HL_merged.txt

# Merge all LH modules
!find ../output/manual_clustering/cbai_transcriptomev2.0/Crab_I -maxdepth 1 -name cluster_LH*txt | xargs -n 1 tail -n +2 > ../output/manual_clustering/cbai_transcriptomev2.0/Crab_I/merged_modules/LH_merged.txt

# Merge all MIX modules
!find ../output/manual_clustering/cbai_transcriptomev2.0/Crab_I -maxdepth 1 -name cluster_MIX*txt | xargs -n 1 tail -n +2 > ../output/manual_clustering/cbai_transcriptomev2.0/Crab_I/merged_modules/MIX_merged.txt

# Won't merge HH or LL modules, as none are present in this crab

Check we did this right by examining number of lines. There will be slightly fewer in merged_modules, as we removed headers

In [44]:
!wc -l ../output/manual_clustering/cbai_transcriptomev2.0/Crab_I/cluster_*txt

  6774 ../output/manual_clustering/cbai_transcriptomev2.0/Crab_I/cluster_HL.txt
  4706 ../output/manual_clustering/cbai_transcriptomev2.0/Crab_I/cluster_LH.txt
    11 ../output/manual_clustering/cbai_transcriptomev2.0/Crab_I/cluster_MIX.txt
 11491 total


In [45]:
!wc -l ../output/manual_clustering/cbai_transcriptomev2.0/Crab_I/merged_modules/*merged.txt

  6773 ../output/manual_clustering/cbai_transcriptomev2.0/Crab_I/merged_modules/HL_merged.txt
  4705 ../output/manual_clustering/cbai_transcriptomev2.0/Crab_I/merged_modules/LH_merged.txt
    10 ../output/manual_clustering/cbai_transcriptomev2.0/Crab_I/merged_modules/MIX_merged.txt
 11488 total


Looks good! We can move on.

## Done merging

Now, let's get a count of the number of lines in each module in each crab

## Line Counts of Modules

In [46]:
!wc -l ../output/manual_clustering/cbai_transcriptomev2.0/Crab_*/merged_modules/*merged.txt

     37 ../output/manual_clustering/cbai_transcriptomev2.0/Crab_A/merged_modules/HLH_merged.txt
   5591 ../output/manual_clustering/cbai_transcriptomev2.0/Crab_A/merged_modules/HTL_merged.txt
   3333 ../output/manual_clustering/cbai_transcriptomev2.0/Crab_A/merged_modules/LHL_merged.txt
  27122 ../output/manual_clustering/cbai_transcriptomev2.0/Crab_A/merged_modules/LTH_merged.txt
     64 ../output/manual_clustering/cbai_transcriptomev2.0/Crab_A/merged_modules/MIX_merged.txt
   1289 ../output/manual_clustering/cbai_transcriptomev2.0/Crab_B/merged_modules/HLH_merged.txt
   4534 ../output/manual_clustering/cbai_transcriptomev2.0/Crab_B/merged_modules/HTL_merged.txt
  10047 ../output/manual_clustering/cbai_transcriptomev2.0/Crab_B/merged_modules/LHL_merged.txt
   4629 ../output/manual_clustering/cbai_transcriptomev2.0/Crab_B/merged_modules/LTH_merged.txt
   2364 ../output/manual_clustering/cbai_transcriptomev2.0/Crab_C/merged_modules/HLH_merged.txt
   4962 ../output/manual_clustering/cbai

We'll now write the above word counts to a file, which we'll then turn into a table using R

In [47]:
!wc -l ../output/manual_clustering/cbai_transcriptomev2.0/Crab_*/merged_modules/*merged.txt > ../output/manual_clustering/cbai_transcriptomev2.0/merged_modules_raw_counts.txt